In [ ]:
import sys
import os
os.environ['OPP4SAR_DIR'] = '/home/jhewers/Documents/meng_project/code/'
sys.path.insert(0,os.getenv('OPP4SAR_DIR'))
DIR = os.path.join(os.getenv('OPP4SAR_DIR'),'generated_data/performance')

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams.update({
    'axes.grid':False,
    'figure.dpi':300,
    'image.cmap':'gray',
    'legend.loc':'upper left',
    'legend.fancybox':True,
    'savefig.bbox':'tight'
    })
def legend_decorator(method):
    def decorate_legend(*args,**kwargs):
        cust_kwargs = {'bbox_to_anchor':(1.05, 1)}
        cust_kwargs.update(kwargs)
        return method(*args,**cust_kwargs)
    return decorate_legend
plt.legend = legend_decorator(plt.legend)

In [ ]:
import json
import numpy as np

from src.json_helpers import GlobalJsonDecoder
from src.enums import WaypointAlgorithmEnum

In [ ]:
with open(os.path.join(DIR, 'output_sim.json'),'r') as f:
    data = json.load(f,cls=GlobalJsonDecoder)
    vehicle = [f[1] for f in data.data if f[0] is WaypointAlgorithmEnum.LHC_GW_CONV][0]
    
    pos = vehicle.pos.toNumpyArray()
    dpos = vehicle.dpos.toNumpyArray()
    ddpos = vehicle.ddpos.toNumpyArray()
    t = np.array(vehicle.t)
    
    del data,vehicle
    
with open(os.path.join(DIR, 'output_wp.json'),'r') as f:
    data = json.load(f,cls=GlobalJsonDecoder)
    wps = data.data[str(WaypointAlgorithmEnum.LHC_GW_CONV)]['wps'].toNumpyArray()
   
    del data
    

pos_bak = pos
dpos_bak = dpos
ddpos_bak = ddpos
t_bak = t
wps_bak = wps

In [ ]:
fig,ax = plt.subplots()
ax.plot(pos[:,0],pos[:,1])
ax.scatter(wps[:,0],wps[:,1],color='b')

In [ ]:
mnx,mxx =[435,475]
mny,mxy = [525,570]

ind = np.where((pos_bak[:,0]>mnx) & (pos_bak[:,0]<mxx) & (pos_bak[:,1]>mny) & (pos_bak[:,1]<mxy))[0]
ind_wps = np.where((wps_bak[:,0]>mnx) & (wps_bak[:,0]<mxx) & (wps_bak[:,1]>mny) & (wps_bak[:,1]<mxy))[0]

pos = pos_bak[ind]
dpos = dpos_bak[ind]
t = t_bak[ind]
wps = wps_bak[ind_wps][::-1]
 
norm_dpos = np.linalg.norm(dpos,axis=1)
min_dpos = dpos[np.where(norm_dpos==np.min(norm_dpos))][0]
max_dpos = dpos[np.where(norm_dpos==np.max(norm_dpos))][0]
min_dpos_pos = pos[np.where(norm_dpos==np.min(norm_dpos))][0]

fig = plt.figure()

### POS ###

ax = plt.subplot(211)

ax.plot(pos[:,0],pos[:,1],label='Sim path')

ax.scatter([],[])
pts = []
for i,wp in enumerate(wps):
    pts.append(ax.scatter(wp[0],wp[1]))

ax.set_ylabel('y (m)')
ax.set_xlabel('x (m)')
ax.set_ylim([mny,mxy])
ax.set_xlim([mnx,mxx])

ax.legend(loc='lower left',bbox_to_anchor=(0,0))

### DPOS ###
ax = plt.subplot(212)

ax.plot(t,norm_dpos,label=r'$| \mathbf{v} |$')
ax.legend(loc='lower left',bbox_to_anchor=(0,0))

ax.set_ylabel(r'$ms^{-1}$')
ax.set_xlabel(r't ($s$)')

a,b,c = wps
y = (-0.1+np.min(norm_dpos),0.1+np.max(norm_dpos))

ax.plot((t[np.where(np.isclose(pos[:,0],a[0],rtol=1e-3) & np.isclose(pos[:,1],a[1],rtol=1e-3))][0],)*2,y,'-.')

ax.plot((t[np.where(norm_dpos==np.min(norm_dpos))],)*2,y,'-.',zorder=-1)

ax.plot((t[np.where(np.isclose(pos[:,0],c[0],rtol=1e-3) & np.isclose(pos[:,1],c[1],rtol=1e-3))][0],)*2,y,'-.')

### MAIN PLOT ###

plt.tight_layout()
fig.legend(pts,['WP 1','WP 2','WP 3'],bbox_to_anchor=(0.95,0.965))
plt.plot()
fig.savefig(os.path.join(DIR,'squarecorner.png'))

In [ ]:
mnx,mxx =[650,690]
mny,mxy = [660,700]

ind = np.where((pos_bak[:,0]>mnx) & (pos_bak[:,0]<mxx) & (pos_bak[:,1]>mny) & (pos_bak[:,1]<mxy))[0]
ind_wps = np.where((wps_bak[:,0]>mnx) & (wps_bak[:,0]<mxx) & (wps_bak[:,1]>mny) & (wps_bak[:,1]<mxy))[0]

dpos = dpos_bak[ind]
ddpos = ddpos_bak[ind]
pos = pos_bak[ind]
t = t_bak[ind]
wps = wps_bak[ind_wps][::-1]
 
norm_dpos = np.linalg.norm(dpos,axis=1)
norm_ddpos = np.linalg.norm(ddpos,axis=1)
min_dpos = dpos[np.where(norm_dpos==np.min(norm_dpos))][0]
max_dpos = dpos[np.where(norm_dpos==np.max(norm_dpos))][0]
min_dpos_pos = pos[np.where(norm_dpos==np.min(norm_dpos))][0]

fig = plt.figure()

### POS ###

ax = plt.subplot(211)

ax.plot(pos[:,0],pos[:,1],label='Sim path')

ax.scatter([],[])
pts = []
for i,wp in enumerate(wps):
    pts.append(ax.scatter(wp[0],wp[1]))

ax.set_ylabel('y (m)')
ax.set_xlabel('x (m)')
ax.set_ylim([mny,mxy])
ax.set_xlim([mnx,mxx])

ax.legend(loc='lower left',bbox_to_anchor=(0,0))

### DPOS ###
ax = plt.subplot(212)

ax.plot(t,norm_dpos,label=r'$| \mathbf{v} |$')
ax.legend(loc='lower left',bbox_to_anchor=(0,0))

ax.set_ylabel(r'$ms^{-1}$')
ax.set_xlabel(r't ($s$)')

a,b = wps
y = (-0.1+np.min(norm_dpos),0.1+np.max(norm_dpos))

ax.plot((t[(np.diff(np.sign(np.diff(norm_dpos))) > 0).nonzero()[0]],)*2,y,'-.')

### MAIN PLOT ###

plt.tight_layout()
fig.legend(pts,['WP 1','WP 2','WP 3'],bbox_to_anchor=(0.95,0.965))
plt.plot()
fig.savefig(os.path.join(DIR,'sharpcorners.png'))

In [ ]:
mnx,mxx =[215,300]
mny,mxy = [100,110]

ind = np.where((pos_bak[:,0]>mnx) & (pos_bak[:,0]<mxx) & (pos_bak[:,1]>mny) & (pos_bak[:,1]<mxy))[0]
ind_wps = np.where((wps_bak[:,0]>mnx) & (wps_bak[:,0]<mxx) & (wps_bak[:,1]>mny) & (wps_bak[:,1]<mxy))[0]

dpos = dpos_bak[ind]
ddpos = ddpos_bak[ind]
pos = pos_bak[ind]
t = t_bak[ind]
wps = wps_bak[ind_wps][::-1]
 
norm_dpos = np.linalg.norm(dpos,axis=1)
norm_ddpos = np.linalg.norm(ddpos,axis=1)
min_dpos = dpos[np.where(norm_dpos==np.min(norm_dpos))][0]
max_dpos = dpos[np.where(norm_dpos==np.max(norm_dpos))][0]
min_dpos_pos = pos[np.where(norm_dpos==np.min(norm_dpos))][0]

fig = plt.figure()

### POS ###

ax = plt.subplot(211)

ax.plot(pos[:,0],pos[:,1],label='Sim path')

ax.scatter([],[])
pts = []
for i,wp in enumerate(wps):
    pts.append(ax.scatter(wp[0],wp[1]))

ax.set_ylabel('y (m)')
ax.set_xlabel('x (m)')
ax.set_ylim([mny,mxy])
ax.set_xlim([mnx,mxx])

ax.legend(loc='lower left',bbox_to_anchor=(0,0))

### DPOS ###
ax = plt.subplot(212)

ax.plot(t,norm_dpos,label=r'$| \mathbf{v} |$')
ax.legend(loc='lower left',bbox_to_anchor=(0,0))

ax.set_ylabel(r'$ms^{-1}$')
ax.set_xlabel(r't ($s$)')

a,b,c = wps
y = (-0.1+np.min(norm_dpos),0.1+np.max(norm_dpos))

for wp in wps:
    ax.plot((t[np.where(np.isclose(pos[:,0],wp[0],rtol=5e-3) & np.isclose(pos[:,1],wp[1],rtol=5e-3))][0],)*2,y,'-.')

### MAIN PLOT ###

plt.tight_layout()
fig.legend(pts,['WP 1','WP 2','WP 3'],bbox_to_anchor=(0.95,0.965))
plt.plot()
fig.savefig(os.path.join(DIR,'straightline.png'))